# Data Cleaning

### Configuración

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from repo_funciones import *


### Adecuación archivo censo

Buscamos homogenizar los valores de la columna fuentes para reducir el numero de opciones y poder agruparlos. 
Empezamos con la premisa de que si tu fuente no es del servicio publico (i.e agua corriente) pasas a disponibilidad no.
Vemos hay muchos valores unicos que se pueden agrupar como por ejemplo 'de una pipa' y 'la trae una pipa'.
Esto nos dará visuales mas interesantesen BI.

In [3]:
#censo=pd.read_csv('../DATA/censo_clean.csv', encoding='latin-1')

In [ ]:
type(censo)

pandas.core.frame.DataFrame

In [ ]:
col_unique_counts(censo, 'fuente')

,fuente,Count
0,No especificado,32
1,Del servicio público de agua,16
2,De un pozo comunitario,16
3,De un pozo particular,16
4,De una pipa,16
5,De otra vivienda,16
6,De la lluvia,16
7,De otro lugar,16
8,Acarreada de un pozo,16
9,Acarreada de llave comunitaria,16


In [ ]:
censo.loc[censo['fuente']=='De una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='La trae una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='Acarreada de otra vivienda' , 'fuente']='De otra vivienda'
censo.loc[censo['fuente']=='La recolectan de la lluvia' , 'fuente']='De la lluvia'
censo.loc[censo['fuente']=='De otro lugar' , 'fuente']='No especificado'
censo.loc[censo['fuente']=='Acarreada de un pozo' , 'fuente']='De un pozo comunitario'
censo.loc[censo['fuente']=='Acarreada de llave comunitaria' , 'fuente']='Llave o pozo comunitario'
censo.loc[censo['fuente']=='De un pozo comunitario' , 'fuente']='Llave o pozo comunitario'

In [ ]:
censo.loc[censo['fuente']=='De una pipa' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Camion cisterna' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De otra vivienda' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De la lluvia' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='No especificado' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Llave o pozo comunitario' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De un pozo particular' , 'disponibilidad']='no'

In [ ]:
censo_group=censo.groupby(['alcaldia','disponibilidad', 'fuente'])['poblacion'].sum()


In [ ]:
censo_grouped=pd.DataFrame(censo_group)

In [ ]:
censo_grouped.head(10)

poblacion
alcaldia      disponibilidad fuente                                 
Azcapotzalco  no             Camion cisterna                    2345
                             De la lluvia                         15
                             De otra vivienda                    293
                             De un pozo particular                65
                             Llave o pozo comunitario            860
                             No especificado                     525
              si             Del servicio público de agua     426697
Benito Juárez no             Camion cisterna                     360
                             De la lluvia                          8
                             De otra vivienda                     67

In [ ]:
censo_grouped.to_csv('../DATA/censo_grouped.csv')

### Adecuación archivo captación

Tomamos el archivo de proyectos de captación de lluvias del proyecto ETL. Recopila los proyectos de 2022. 
A este primero le tendremos que añadir una columna de año ya que le vamos a añadir los archivos respectivos de 19 a 21. 
Esto porque nos interesa medir la evolución de proyectos/capacidad/alacladías a través del tiempo. 

De igual forma tendremos que igualar el orden de las columnas para que cuadren a la hora de concatenar. 

In [75]:
captacion_22=pd.read_csv('../DATA/CLEAN/captacion_clean.csv')

In [76]:
captacion_22['año']=2022

In [77]:
captacion_22.head()

,capacidad,alcaldia,latitud,longitud,alcaldia_id,año
0,2500,Milpa Alta,19.219028,-98.991283,9,2022
1,2500,Milpa Alta,19.219880,-99.002227,9,2022
2,2500,Milpa Alta,19.218343,-98.997507,9,2022
3,2500,Milpa Alta,19.219277,-98.996269,9,2022
4,2500,Milpa Alta,19.222071,-98.997760,9,2022


In [78]:
col_unique_counts(captacion_22, 'alcaldia')

,alcaldia,Count
0,Milpa Alta,8750
1,Tlalpan,3016
2,Tláhuac,2373
3,Iztapalapa,1360
4,Gustavo A. Madero,916
5,Xochimilco,556


In [79]:
#Borramos columna capacidad, ya que tras hechar un ojo a los archivos de años pasados, niguno trae esta información.
captacion_22.drop('capacidad', axis=1, inplace=True)


In [80]:
captacion_19=pd.read_csv('../DATA/RAW/scall_2019.csv')

In [81]:
captacion_19.info() #vemos que este trae la indo de alcaldía bien llenada no como el de 2022 por lo que nos ahorra tener que limpiar. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10003 entries, 0 to 10002
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   long        10003 non-null  float64
 1   lat         10003 non-null  float64
 2   Expediente  10003 non-null  object 
 3   Pueblo      10003 non-null  object 
 4   Colonia_Ba  10003 non-null  object 
 5   Territoria  10003 non-null  object 
 6   Alcaldia    10003 non-null  object 
 7   Fecha_de_I  10000 non-null  object 
 8   Instalado   10003 non-null  object 
dtypes: float64(2), object(7)
memory usage: 703.5+ KB


In [82]:
captacion_19['Alcaldia'] = captacion_19['Alcaldia'].str.title()


In [130]:
def map_to_alcaldia_id(df, col_name):
    alcaldia_dict = {
        'Azcapotzalco': '002',
        'Coyoacan': '003',
        'Cuajimalpa de Morelos': '004',
        'Gustavo A. Madero': '005',
        'Iztacalco': '006',
        'Iztapalapa': '007',
        'Magdalena Contreras': '008',
        'Milpa Alta': '009',
        'Alvaro Obregon': '010',
        'Tlahuac': '011',
        'Tlalpan': '012',
        'Xochimilco': '013',
        'Benito Juarez': '014',
        'Cuauhtemoc': '015',
        'Miguel Hidalgo': '016',
        'Venustiano Carranza': '017'
    }
    
    df['alcaldia_id'] = df[col_name].map(alcaldia_dict)
    
    return df.head()


In [84]:
map_to_alcaldia_id(captacion_19, 'Alcaldia')

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado,alcaldia_id
0,-98.975686,19.328924,J16NRB,TEOTONGO ACAHUALTEPEC,CAMPESTRE POTRERO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-24,Programa SCALL,007
1,-98.970402,19.327016,J5WU9F,TEOTONGO ACAHUALTEPEC,CAMPESTRE POTRERO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-24,Programa SCALL,007
2,-98.969184,19.326854,X4S7RV,TEOTONGO ACAHUALTEPEC,SAN FRANCISCO APOLOCALCO,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-23,Programa SCALL,007
3,-98.995963,19.334764,F5AXVR,TEOTONGO ACAHUALTEPEC,MIRAVALLES,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-07-25,Programa SCALL,007
4,-99.006770,19.353941,M8IJD1,TEOTONGO ACAHUALTEPEC,SANTIAGO ACAHUALTEPEC 1a amp,TEOTONGO ACAHUALTEPEC,Iztapalapa,2019-06-06,Programa SCALL,007


In [85]:
col_unique_counts(captacion_19, 'Alcaldia')

,Alcaldia,Count
0,Iztapalapa,5833
1,Xochimilco,4170


In [86]:
col_unique_counts(captacion_19, 'alcaldia_id')
#para checar cuadra con los valores unicos de arriba y que la formula custom funcionó bien

,alcaldia_id,Count
0,007,5833
1,013,4170


In [87]:
captacion_19.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)

In [88]:
captacion_19['año']=2019

In [89]:
captacion_19 = captacion_19.rename(columns={'Alcaldia': 'alcaldia'})

In [90]:
captacion_19 = captacion_19.rename(columns={'long': 'longitud'})
captacion_19 = captacion_19.rename(columns={'lat': 'latitud'})

In [91]:
captacion_19.head()

,longitud,latitud,alcaldia,alcaldia_id,año
0,-98.975686,19.328924,Iztapalapa,007,2019
1,-98.970402,19.327016,Iztapalapa,007,2019
2,-98.969184,19.326854,Iztapalapa,007,2019
3,-98.995963,19.334764,Iztapalapa,007,2019
4,-99.006770,19.353941,Iztapalapa,007,2019


In [92]:
captacion_19 = captacion_19.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [93]:
captacion_19.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Iztapalapa,-98.975686,19.328924,007,2019
1,Iztapalapa,-98.970402,19.327016,007,2019
2,Iztapalapa,-98.969184,19.326854,007,2019
3,Iztapalapa,-98.995963,19.334764,007,2019
4,Iztapalapa,-99.006770,19.353941,007,2019


In [23]:
#19 listo, proximo año

In [114]:
captacion_20=pd.read_csv('../DATA/RAW/scall_2020.csv')

In [115]:
captacion_20.head()

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado
0,-99.101716,19.348128,3-VESAJO-IZA-20,NO APLICA,VALLE DE LUCES,LOS CULHUACANES,IZTAPALAPA,2020-08-18,Programa SCALL
1,-99.101154,19.375799,11-ARCAKA-IZA-20,NO APLICA,SAN JOSE ACULCO,ACULCO,IZTAPALAPA,2020-10-03,Programa SCALL
2,-99.051573,19.319776,20-PAMAMA-IZA-20,NO APLICA,EL TRIANGULO,SANTA CRUZ QUETZALCOATL,IZTAPALAPA,2020-11-18,Programa SCALL
3,-99.071098,19.374533,23-COLOLU-IZA-20,NO APLICA,LEYES DE REFORMA 3ERA SECCION,LEYES DE REFORMA,IZTAPALAPA,2020-09-25,Programa SCALL
4,-99.025270,19.326348,30-SASONO-IZA-20,NO APLICA,BUENAVISTA,SANTA CATARINA,IZTAPALAPA,2020-10-02,Programa SCALL


In [116]:
captacion_20['Alcaldia'] = captacion_20['Alcaldia'].str.title()
map_to_alcaldia_id(captacion_20, 'Alcaldia')
captacion_20.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)
captacion_20['año']=2020
captacion_20 = captacion_20.rename(columns={'Alcaldia': 'alcaldia'})
captacion_20 = captacion_20.rename(columns={'long': 'longitud'})
captacion_20 = captacion_20.rename(columns={'lat': 'latitud'})
captacion_20 = captacion_20.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [117]:
captacion_20.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Iztapalapa,-99.101716,19.348128,007,2020
1,Iztapalapa,-99.101154,19.375799,007,2020
2,Iztapalapa,-99.051573,19.319776,007,2020
3,Iztapalapa,-99.071098,19.374533,007,2020
4,Iztapalapa,-99.025270,19.326348,007,2020


In [118]:
nancols(captacion_20)

Series([], dtype: int64)

In [131]:
captacion_21=pd.read_csv('../DATA/RAW/scall_2021.csv')

In [132]:
captacion_21.head()

,long,lat,Expediente,Pueblo,Colonia_Ba,Territoria,Alcaldia,Fecha_de_I,Instalado
0,-99.186038,19.493667,2-MAMUNO-AZO-21,NO APLICA,PUEBLO SAN ANDRES,NO APLICA,AZCAPOTZALCO,2021-05-25,Programa SCALL
1,-99.219528,19.475549,4-MASAHO-AZO-21,NO APLICA,AMPL. SAN PEDRO XALPA,NO APLICA,AZCAPOTZALCO,2021-09-02,Programa SCALL
2,-99.211189,19.480327,5-LOLOMA-AZO-21,NO APLICA,PUEBLO SANTIAGO AHUIZOTLA,NO APLICA,AZCAPOTZALCO,2021-05-10,Programa SCALL
3,-99.176603,19.486284,6-SAARJO-AZO-21,NO APLICA,SANTO TOMAS,NO APLICA,AZCAPOTZALCO,2021-05-27,Programa SCALL
4,-99.218356,19.481075,12-PEALAR-AZO-21,NO APLICA,AMPL. SAN PEDRO XALPA,NO APLICA,AZCAPOTZALCO,2021-08-25,Programa SCALL


In [133]:
captacion_21['Alcaldia'] = captacion_21['Alcaldia'].str.title()
map_to_alcaldia_id(captacion_21, 'Alcaldia')
captacion_21.drop(columns=['Expediente', 'Colonia_Ba', 'Territoria', 'Fecha_de_I', 'Instalado', 'Pueblo'], inplace=True)
captacion_21['año']=2021
captacion_21 = captacion_21.rename(columns={'Alcaldia': 'alcaldia'})
captacion_21 = captacion_21.rename(columns={'long': 'longitud'})
captacion_21 = captacion_21.rename(columns={'lat': 'latitud'})
captacion_21 = captacion_21.reindex(columns=['alcaldia', 'longitud', 'latitud', 'alcaldia_id', 'año'])

In [134]:
captacion_21.head()

,alcaldia,longitud,latitud,alcaldia_id,año
0,Azcapotzalco,-99.186038,19.493667,002,2021
1,Azcapotzalco,-99.219528,19.475549,002,2021
2,Azcapotzalco,-99.211189,19.480327,002,2021
3,Azcapotzalco,-99.176603,19.486284,002,2021
4,Azcapotzalco,-99.218356,19.481075,002,2021


In [135]:
nancols(captacion_21)

Series([], dtype: int64)

In [136]:
#exportamos...luego hacemos concat
captacion_21.to_csv('../DATA/CLEAN/captacion21_clean.csv', index=False)
captacion_22.to_csv('../DATA/CLEAN/captacion22_clean.csv', index=False)
captacion_20.to_csv('../DATA/CLEAN/captacion20_clean.csv', index=False)
captacion_19.to_csv('../DATA/CLEAN/captacion19_clean.csv', index=False)

Momento de concatenar el historico de 2019 hasta 2022

In [137]:
captacion_total=pd.concat([captacion_22,captacion_21,captacion_20,captacion_19], axis= 0)

In [104]:
captacion_22.shape, captacion_21.shape, captacion_20.shape, captacion_19.shape

((16971, 5), (10015, 5), (10007, 5), (10003, 5))

In [105]:
captacion_total.shape

(46996, 5)

In [138]:
captacion_total.to_csv('../DATA/CLEAN/captacion_total_clean.csv', encoding='latin-1', index=False)

### Adecuación archivo consumo por colonia

Este archivo viene desagregado por bimestre y por manzanas dentro de cada columna. Resultando en + de 70 filas de archivo...

A nivel de columnas, traen unos promedios que no sabemos si es por vivienda o por habitantes o cual es la medida. En la documentación tampoco viene mención. Por lo tanto nos quedaremos unicamente con la columna de consumo total domestico y haremos un cruce con poblacion por alcaldia para sacar el promedio de uso por alcaldia. 

Para efectos de este analisis nos estamos centrando en escala alcaldía y tomaremos el total bimestre.

In [36]:
consumo= pd.read_csv('../DATA/RAW/consumo_agua_historico_2019.csv')

In [37]:
consumo.drop(columns=['consumo_prom_dom', 'consumo_prom_mixto', 'consumo_prom', 'consumo_prom_no_dom', 'indice_des', 'colonia', 'latitud', 'longitud', 'fecha_referencia', 'anio', 'bimestre' ], inplace=True)

In [38]:
consumo.head()

,consumo_total_mixto,consumo_total_dom,consumo_total,consumo_total_no_dom,alcaldia
0,159.72,468.23,631.00,3.05,GUSTAVO A. MADERO
1,0.00,107.81,115.13,7.32,GUSTAVO A. MADERO
2,0.00,122.93,197.96,75.03,GUSTAVO A. MADERO
3,0.00,0.00,253.53,253.53,GUSTAVO A. MADERO
4,56.72,539.49,839.35,243.14,AZCAPOTZALCO


In [23]:
consumo.shape

(71102, 5)

In [40]:
consumo['alcaldia'] = consumo['alcaldia'].str.title()

In [41]:
consumo_alc=consumo.groupby('alcaldia')[['consumo_total_mixto', 'consumo_total_dom', 'consumo_total', 'consumo_total_no_dom']].sum()


In [42]:
consumo_alc.shape

(16, 4)

In [48]:
consumo_alc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, Alvaro Obregon to Xochimilco
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   consumo_total_mixto   16 non-null     float64
 1   consumo_total_dom     16 non-null     float64
 2   consumo_total         16 non-null     float64
 3   consumo_total_no_dom  16 non-null     float64
dtypes: float64(4)
memory usage: 640.0+ bytes


In [50]:
consumo_alc=consumo_alc.reset_index()

#grouped_df = grouped_df.reset_index()

In [51]:
consumo_alc

,alcaldia,consumo_total_mixto,consumo_total_dom,consumo_total,consumo_total_no_dom
0,Alvaro Obregon,525476.88,6.590161e+06,9.608545e+06,2492906.660
1,Azcapotzalco,918808.67,6.153755e+06,8.969567e+06,1897004.540
2,Benito Juarez,1069917.90,9.894931e+06,1.378012e+07,2815269.571
3,Coyoacan,350036.32,5.128303e+06,7.687908e+06,2209569.070
4,Cuajimalpa De Morelos,147131.73,2.064264e+06,2.945011e+06,733614.840
5,Cuauhtemoc,2214262.43,1.020769e+07,1.840177e+07,5979812.391
6,Gustavo A. Madero,1542992.79,9.196449e+06,1.348010e+07,2740658.660
7,Iztacalco,687157.63,4.663498e+06,6.260631e+06,909975.230
8,Iztapalapa,837352.80,5.263027e+06,8.313478e+06,2213097.790
9,La Magdalena Contreras,65018.72,8.449665e+05,1.271478e+06,361492.280


In [61]:
data = {
    'alcaldia_id': ['002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017'],
    'alcaldia': ['Azcapotzalco', 'Coyoacan', 'Cuajimalpa de Morelos', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras', 'Milpa Alta', 'Alvaro Obregon', 'Tlahuac', 'Tlalpan', 'Xochimilco', 'Benito Juarez', 'Cuauhtemoc', 'Miguel Hidalgo', 'Venustiano Carranza'],
    'poblacion': [432205, 614447, 217686, 1173351, 404695, 1835486, 247622, 152685, 759137, 392313, 699928, 442178, 434153, 545884, 414470, 443704]
}

pob_alc = pd.DataFrame(data)

pob_alc

#esta tabla va a servirnos para sacar el promedio de consumo domestico usandpo estos datos de poblacion 


,alcaldia_id,alcaldia,poblacion
0,002,Azcapotzalco,432205
1,003,Coyoacan,614447
2,004,Cuajimalpa de Morelos,217686
3,005,Gustavo A. Madero,1173351
4,006,Iztacalco,404695
5,007,Iztapalapa,1835486
6,008,La Magdalena Contreras,247622
7,009,Milpa Alta,152685
8,010,Alvaro Obregon,759137
9,011,Tlahuac,392313


In [62]:
merged_df = pd.merge(consumo_alc, pob_alc[['alcaldia', 'poblacion']], on='alcaldia', how='left')

In [64]:
merged_df.loc[merged_df['alcaldia']=='Cuajimalpa De Morelos', 'poblacion']=217686

In [68]:
merged_df['consumo_prom_dom'] = merged_df['consumo_total_dom'] / merged_df['poblacion']


In [69]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   alcaldia              16 non-null     object 
 1   consumo_total_mixto   16 non-null     float64
 2   consumo_total_dom     16 non-null     float64
 3   consumo_total         16 non-null     float64
 4   consumo_total_no_dom  16 non-null     float64
 5   poblacion             16 non-null     float64
 6   consumo_prom_dom      16 non-null     float64
dtypes: float64(6), object(1)
memory usage: 1.0+ KB


In [71]:
merged_df

,alcaldia,consumo_total_mixto,consumo_total_dom,consumo_total,consumo_total_no_dom,poblacion,consumo_prom_dom
0,Alvaro Obregon,525476.88,6.590161e+06,9.608545e+06,2492906.660,759137.0,8.681123
1,Azcapotzalco,918808.67,6.153755e+06,8.969567e+06,1897004.540,432205.0,14.238046
2,Benito Juarez,1069917.90,9.894931e+06,1.378012e+07,2815269.571,434153.0,22.791345
3,Coyoacan,350036.32,5.128303e+06,7.687908e+06,2209569.070,614447.0,8.346209
4,Cuajimalpa De Morelos,147131.73,2.064264e+06,2.945011e+06,733614.840,217686.0,9.482760
5,Cuauhtemoc,2214262.43,1.020769e+07,1.840177e+07,5979812.391,545884.0,18.699377
6,Gustavo A. Madero,1542992.79,9.196449e+06,1.348010e+07,2740658.660,1173351.0,7.837765
7,Iztacalco,687157.63,4.663498e+06,6.260631e+06,909975.230,404695.0,11.523487
8,Iztapalapa,837352.80,5.263027e+06,8.313478e+06,2213097.790,1835486.0,2.867375
9,La Magdalena Contreras,65018.72,8.449665e+05,1.271478e+06,361492.280,247622.0,3.412324


In [72]:
merged_df.to_csv('../DATA/CLEAN/consumo_alc.csv', index=False)